In [116]:
import pandas as pd
import plotly.express as px
from plotly.offline import init_notebook_mode
init_notebook_mode(connected=True)

In [117]:
data = pd.read_csv('atendimentos_2023_clean.csv')


In [118]:
data.columns

Index(['Atendimento Classificação', 'etapa', 'Atendimento código',
       'Atendimento comissão', 'convertido', 'Atendimento data de Conclusão',
       'data_inicial', 'Atendimento data de criação Full',
       'Atendimento data de Início', 'Atendimento data de Início Full',
       'Atendimento dias sem interação do usuário',
       'Atendimento email integração', 'motivo', 'encerrado', 'valor_negocio',
       'Campanha', 'Canal', 'Canal Meio', 'Grupo', 'Grupo Hierarquia',
       'Interação Usuário Abreviado (última)',
       'Interação Usuário Data (primeira)', 'Interação Usuário Data (última)',
       'Interação Usuário qtd dias (última)',
       'Interação Usuário Tipo (última)', 'source', 'produto', 'Produto Sub',
       'Produto UF'],
      dtype='object')

In [119]:
# Columns to examine
columns = ['etapa', 'encerrado', 'convertido', "motivo", 'produto', 'valor_negocio', 'source']
data = data[columns]

In [120]:
data.describe(include='all')

,etapa,encerrado,convertido,motivo,produto,valor_negocio,source
count,449,449,449,393,449,449.000000,438
unique,4,2,2,11,3,NaN,7
top,Pesquisa,ENCERRADO,NÃO CONVERTIDO,Sem interesse - Outros,Quinta do Lago,NaN,Facebook
freq,444,393,444,196,251,NaN,154
mean,NaN,NaN,NaN,NaN,NaN,2897.316258,NaN
std,NaN,NaN,NaN,NaN,NaN,27478.794941,NaN
min,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
25%,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
50%,NaN,NaN,NaN,NaN,NaN,0.000000,NaN
75%,NaN,NaN,NaN,NaN,NaN,0.000000,NaN


In [121]:
# Check the unique values of source column
data.source.unique()

array(['Facebook', 'Vivareal', 'Zap', 'Site', nan, 'ZapImoveis', 'Google',
       'Viva Real'], dtype=object)

In [122]:
# Viva Real and Vivareal are the same source
# Let's fix it
data.source = data.source.replace('Vivareal', 'Viva Real')

In [123]:
data.source.unique()

array(['Facebook', 'Viva Real', 'Zap', 'Site', nan, 'ZapImoveis',
       'Google'], dtype=object)

In [124]:
# Zap, Zap Imoveis and ZapImoveis are the same source
# Let's fix it
data.source = data.source.replace('Zap Imoveis', 'Zap')
data.source = data.source.replace('ZapImoveis', 'Zap')

In [125]:
data.source.unique()

array(['Facebook', 'Viva Real', 'Zap', 'Site', nan, 'Google'],
      dtype=object)

In [126]:
# We have a NaN value in source column
# Let's check it how many rows have NaN value
data.source.isna().sum()
data.source.isna().sum() / data.shape[0]

0.024498886414253896

2% of rows in source column have NaN.

In [127]:
# Drop nan values
data = data.dropna(subset=['source'])

In [128]:
data.source.isna().sum()

0

Funil de marketing. Quais etapas da jornada foram registradas no CRM.

In [129]:
# The marketing funnel
mkt_funnel = data.etapa.value_counts(normalize=True).sort_values(ascending=False)

In [130]:
mkt_funnel

Pesquisa    0.990868
Visita      0.006849
Proposta    0.002283
Name: etapa, dtype: float64

In [131]:
funnel_data = dict(
    number=mkt_funnel.values,
    stage=mkt_funnel.index)
fig = px.funnel(funnel_data, x='number', y='stage', 
                color='stage')
fig.show()

Existe uma grande diferença entre as etapas, o que levanta uma suspeita em como estas informações estão sendo alimentadas no CRM pelas pessoas.

Aproveitamento da base

In [132]:
conversion = data.convertido.value_counts(normalize=True)

In [133]:
conversion

NÃO CONVERTIDO    0.990868
CONVERTIDO        0.009132
Name: convertido, dtype: float64

Menos de 1% dos leads válidos foram convertidos em venda.
Este seria o cenário recorrente?

In [134]:
# Where the leads come from?
source_of_leads = data.source.value_counts(normalize=True).sort_values(ascending=False)
# plot
fig = px.bar(source_of_leads, x=source_of_leads.index, y=source_of_leads.values,
                labels={'x': 'Canal', 'y': '% de leads'},
                title='Leads gerados por canal',
                color=source_of_leads.index)
fig.show()


Qual os motivos pela não ocnversão?

In [135]:
# Let's find the non conversion
# Look the motivos column
data.motivo.unique()

array(['Sem interesse - Outros', 'Falta de interação do corretor', nan,
       'Contato Inexistente', 'Sem interesse - Localização',
       'Sem interesse - Preço', 'Sem interesse - Comprou imóvel recente',
       'Corretor', 'Venda realizada', 'Sem interesse - Metragem ',
       'Pesquisa e Ofertas', 'Cliente proprietário'], dtype=object)

In [136]:
# Plot the non conversion
non_conversion = data[data.convertido == 'NÃO CONVERTIDO'].motivo.value_counts(normalize=True).sort_values(ascending=False)

non_conversion
# pizza plot
fig = px.pie(non_conversion, values=non_conversion.values, names=non_conversion.index,
                title='Motivos de não conversão')
fig.show()


Qual canal obteve o melhor aproveitamento?

In [137]:
# What the value per source?
source_value = data.groupby('source').valor_negocio.sum().sort_values(ascending=False)

# Plotting
fig = px.bar(source_value, x=source_value.index, y=source_value.values,
                color=source_value.index,
                title='Vendas por canal',
                labels={'x':'Canal', 'y':'Valor'})
fig.show()

Viva Real foi o segundo canal em % de leads gerados, mas trouxe o maior volume de vendas.

Facebook e Google resultaram volume vendas semelhantes. Porém o Google parece ter o melhor aproveitamento, já que trouxe o menor número de leads entre os canais analisados. 

#### Qual canal teve o melhor ROAS (return on ads spend)?


In [138]:
# Create Investimento series
investimento = {'Facebook': 17324, 'Google': 1970, 'Instagram': 17324, 'Viva Real': 7500, 'Zap': 7500}
investimento = pd.Series(investimento)


In [139]:
investimento

Facebook     17324
Google        1970
Instagram    17324
Viva Real     7500
Zap           7500
dtype: int64

In [140]:
# Calculate the ROAS
roas = source_value / investimento
roas

Facebook      15.019626
Google       122.842640
Instagram           NaN
Site                NaN
Viva Real     64.933333
Zap            0.000000
dtype: float64

In [141]:
# What's the best roas?
# Include a column with % comparing the best roas with each other
# Replace NaN values with 0
roas = roas.fillna(0)
roas = roas.sort_values(ascending=False)
roas = roas.reset_index()
roas.columns = ['source', 'ROAS']
roas['%'] = roas.ROAS / roas.ROAS.max()
roas

,source,ROAS,%
0,Google,122.842640,1.000000
1,Viva Real,64.933333,0.528590
2,Facebook,15.019626,0.122267
3,Instagram,0.000000,0.000000
4,Site,0.000000,0.000000
5,Zap,0.000000,0.000000


In [142]:
# Plotting
fig = px.bar(roas, x=roas.source, y=roas.ROAS,
                color=roas.source,
                title='ROAS por canal',
                labels={'x':'Canal', 'y':'ROAS'})
fig.show()


Google obteve (de longe) o melhor ROAS com 122.

Viva Real obteve 50% do ROAS entregue pelo Google.

Já o Facebook (e Instagram) responderam por apenas 12% do ROAS entregue pelo Google.
